**Rationale** This assignment will help you practice running and interpreting basic regressions analysis.

**[Datasets](https://drive.google.com/drive/folders/1LJ38ctVX62ggGGygJDLlLG2V6jZ4BNml?usp=sharing)**
1. [Safegraph Temple Visitor Sample](https://drive.google.com/file/d/1u_sDxFp9fja-_cop4yzIAJ_hX2-fplWN/view?usp=sharing) 
1. [Avocado Prices](https://drive.google.com/file/d/1QJ_dyyBFkHhC6VkzVvdAFa385mtqtntM/view?usp=sharing)

**Data description** This dataset includes SafeGraph's sampled foot traffic volume for Temple University between 1-1-2019 and 5-31-2020. Safegraph collects the foot traffic data from a network of phone apps. You can view the `visits` data as the number of SafeGraph tracked devices to visit Temple University. This dataset is merged with Philadelphia's weather data. 


# 1. Run and interpret a simple linear regression (5 points)...
...to figure out the relationship between daily average temperateure as measured in degrees fahrenheit, `"temp"`, and people on campus `"visits"` for the Fall 2019 semester (limit data to M-F 8-26 - 11-24-2019, inclusive).

In your answer: 
1. First, look at data. What is the average number of SafeGraph visitors to Temple University during the teaching days of the Fall 2019 semester? Roughly what percent of the Temple student population is this?
1. Specify (write out the regression equation), run, and interpret a regression that documents the relationship between temperature and foot traffic (visits) on campus.
    1. Do more students visit campus (and presumably go to class) when the weather is warmer or colder?
    1. Besides students not wanting to go to class when the temperature is not to their liking, what else might cause the observed relationship between visits and temperature?

In [1]:
# imports here
import os, matplotlib.pyplot as plt
import pandas
#from google.colab import drive
from statsmodels.formula import api as smf
#drive.mount('drive')

In [2]:
fpath = !pwd
print(fpath)

['/home/johndoe/assign6']


In [3]:
# list directory containing the data
#fpath = 'drive/MyDrive/Teaching/Analytics/Datasets/A6/' # change this to your data folder
fpath='/home/johndoe/assign6'
os.listdir(fpath)

['temple_traffic.csv',
 'A6_F2021_Regression_1.ipynb',
 '.ipynb_checkpoints',
 'avocado.csv']

In [4]:
# read in the dataset
temple = pandas.read_csv('temple_traffic.csv', index_col = 0)
temple.head(5)

,safegraph_place_id,date,visits,stn,wban,temp,count_temp,dewp,count_dewp,slp,...,mxspd,gust,maxtemp,flag_max,mintemp,flag_min,prcp,flag_prcp,sndp,frshtt
0,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-02,65,997286.0,99999.0,45.8,24.0,NaN,0.0,1016.8,...,NaN,NaN,63.3,*,34.7,*,0.0,I,NaN,0.0
1,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-03,68,997286.0,99999.0,53.0,24.0,NaN,0.0,1006.3,...,NaN,NaN,59.9,*,45.9,*,0.0,I,NaN,0.0
2,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-04,71,997286.0,99999.0,51.4,24.0,NaN,0.0,1006.3,...,NaN,NaN,58.6,*,45.1,*,0.0,I,NaN,0.0
3,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-05,70,997286.0,99999.0,49.2,24.0,NaN,0.0,1017.0,...,NaN,NaN,55.8,*,41.7,*,0.0,I,NaN,0.0
4,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-06,57,997286.0,99999.0,41.7,24.0,NaN,0.0,1013.1,...,NaN,NaN,49.1,*,37.2,*,0.0,I,NaN,0.0


In [5]:
# convert the date column to pandas datetime using pd.to_datetime(...)
# if you've fogotten, look through notes from module 3
temple['date'] = pandas.to_datetime(temple['date'])
temple.head(5)

,safegraph_place_id,date,visits,stn,wban,temp,count_temp,dewp,count_dewp,slp,...,mxspd,gust,maxtemp,flag_max,mintemp,flag_min,prcp,flag_prcp,sndp,frshtt
0,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-02,65,997286.0,99999.0,45.8,24.0,NaN,0.0,1016.8,...,NaN,NaN,63.3,*,34.7,*,0.0,I,NaN,0.0
1,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-03,68,997286.0,99999.0,53.0,24.0,NaN,0.0,1006.3,...,NaN,NaN,59.9,*,45.9,*,0.0,I,NaN,0.0
2,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-04,71,997286.0,99999.0,51.4,24.0,NaN,0.0,1006.3,...,NaN,NaN,58.6,*,45.1,*,0.0,I,NaN,0.0
3,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-05,70,997286.0,99999.0,49.2,24.0,NaN,0.0,1017.0,...,NaN,NaN,55.8,*,41.7,*,0.0,I,NaN,0.0
4,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-06,57,997286.0,99999.0,41.7,24.0,NaN,0.0,1013.1,...,NaN,NaN,49.1,*,37.2,*,0.0,I,NaN,0.0


In [6]:
# create a new column called 'dow' (for day of week) using temple.date.dt.weekday
# note that Monday through Sunday corresponds to 0 through 6
temple['dow'] = temple.date.dt.weekday
temple.head(5)


,safegraph_place_id,date,visits,stn,wban,temp,count_temp,dewp,count_dewp,slp,...,gust,maxtemp,flag_max,mintemp,flag_min,prcp,flag_prcp,sndp,frshtt,dow
0,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-02,65,997286.0,99999.0,45.8,24.0,NaN,0.0,1016.8,...,NaN,63.3,*,34.7,*,0.0,I,NaN,0.0,0
1,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-03,68,997286.0,99999.0,53.0,24.0,NaN,0.0,1006.3,...,NaN,59.9,*,45.9,*,0.0,I,NaN,0.0,1
2,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-04,71,997286.0,99999.0,51.4,24.0,NaN,0.0,1006.3,...,NaN,58.6,*,45.1,*,0.0,I,NaN,0.0,2
3,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-05,70,997286.0,99999.0,49.2,24.0,NaN,0.0,1017.0,...,NaN,55.8,*,41.7,*,0.0,I,NaN,0.0,3
4,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-06,57,997286.0,99999.0,41.7,24.0,NaN,0.0,1013.1,...,NaN,49.1,*,37.2,*,0.0,I,NaN,0.0,4


In [7]:
# select only the rows where the date lies between 2019/8/26 and 2019/11/24 (inclusive) 
# and correspond to a regular school date (e.g. are M-F)
# call the reuslting dataframe F2019
# remember you can select rows using df.loc[conditions]
# If you forgot how to do this, look at the notes and in class exercise from module 3

F2019 = temple.sort_values(by = 'date')
F2019 = F2019[F2019['date'] >= '2019/8/26'] 
F2019 = F2019[F2019['date'] <= '2019/11/24']
F2019 = F2019.loc[(F2019.dow < 5)] 
F2019


,safegraph_place_id,date,visits,stn,wban,temp,count_temp,dewp,count_dewp,slp,...,gust,maxtemp,flag_max,mintemp,flag_min,prcp,flag_prcp,sndp,frshtt,dow
140,sg:00579e722c8e48178ef0c66a7c91f92c,2019-08-26,166,997286.0,99999.0,68.1,23.0,NaN,0.0,1021.9,...,NaN,73.8,*,62.8,*,0.0,I,NaN,0.0,0
141,sg:00579e722c8e48178ef0c66a7c91f92c,2019-08-27,162,997286.0,99999.0,70.1,23.0,NaN,0.0,1018.0,...,NaN,78.3,*,65.1,*,0.0,I,NaN,0.0,1
142,sg:00579e722c8e48178ef0c66a7c91f92c,2019-08-28,172,997286.0,99999.0,73.8,24.0,NaN,0.0,1012.4,...,NaN,82.0,*,68.2,*,0.0,I,NaN,0.0,2
143,sg:00579e722c8e48178ef0c66a7c91f92c,2019-08-29,168,997286.0,99999.0,75.7,24.0,NaN,0.0,1011.8,...,NaN,84.4,*,68.0,*,0.0,I,NaN,0.0,3
144,sg:00579e722c8e48178ef0c66a7c91f92c,2019-08-30,194,997286.0,99999.0,77.5,23.0,NaN,0.0,1016.2,...,NaN,89.8,*,66.4,*,0.0,I,NaN,0.0,4
175,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-02,88,997286.0,99999.0,74.6,24.0,NaN,0.0,1018.5,...,NaN,86.7,*,68.9,*,0.0,I,NaN,0.0,0
176,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-03,155,997286.0,99999.0,76.2,24.0,NaN,0.0,1015.8,...,NaN,84.4,*,69.8,*,0.0,I,NaN,0.0,1
177,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-04,183,997286.0,99999.0,78.2,21.0,NaN,0.0,1014.2,...,NaN,91.8,*,70.5,*,0.0,I,NaN,0.0,2
178,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-05,172,997286.0,99999.0,73.0,24.0,NaN,0.0,1016.3,...,NaN,76.5,*,67.6,*,0.0,I,NaN,0.0,3
179,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-06,130,997286.0,99999.0,67.3,23.0,NaN,0.0,1012.2,...,NaN,72.0,*,62.2,*,0.0,I,NaN,0.0,4


In [8]:
# find the maximum date of F2019, make sure that it is less than or equal to 2019/11/24
max_value = F2019['date'].max()



In [9]:
# find the minimum date of F2019, make sure that it is greater than or equal to 2019/8/26
min_value = F2019['date'].min()
min_value


Timestamp('2019-08-26 00:00:00')

In [10]:
# display the unique values of dow in F2019 (use the .unique() method on dow column)
# make sure there are only M-F (e.g. 0-4)
F2019['dow'].unique()


array([0, 1, 2, 3, 4])

In [11]:
# compute the mean visits
mean = F2019['visits'].mean()
mean

137.73846153846154

In [12]:
# look up the total number of temple students in 2019 and compute the percentage the above average represents
temple_students = 40000
percentage = (mean/temple_students)  * 100
percentage



0.34434615384615386

**Edit this cell**
The average number of visitors in the sample is **137.738**, this is approximately **0.34%** of the Temple student population.

**Edit this cell**

Before running a regression to document the relationship between temperature and the sampled visitors, answer the following.

The logical dependent variable should be **vistors**, the logical indepdent (aka explanatory) variable should be **temperature** because **temperatrue** should cause changes in **visitors** and not the other way around.


Consequently, the following regression equation describes the relationship between temperature and foot traffic (change the $x$ and $y$ to the approriate variable names):

$$
y = \beta_0 +\beta_1x + e 
$$


In [24]:
# run the above regression using statsmodels
# call the resulting regression results object the variable "res"

res = smf.ols('visits ~ temp', data = F2019).fit()


In [25]:
# print the regression summary table (e.g. print(res.summary()))
##print(res.summary())
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 visits   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     55.33
Date:                Tue, 19 Oct 2021   Prob (F-statistic):           3.43e-10
Time:                        21:22:05   Log-Likelihood:                -285.36
No. Observations:                  65   AIC:                             574.7
Df Residuals:                      63   BIC:                             579.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.8564     13.255      3.082      0.003      14.369      67.343
temp           1.5446      0.208      7.438      0.000       1.130       1.960
==============================================================================
Omnibus:                        7.362   Durbin-Watson:                   1.341
Prob(Omnibus):                  0.025   Jarque-Bera (JB):                6.733
Skew:                          -0.639   Prob(JB):                       0.0345
Kurtosis:                       3.922   Cond. No.                         344.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Edit this cell**

The regression's R$^2$ is **0.468**, this suggests that **47%**\% of the variation in **visits** is explained by **temperature**.

The intercept ($\beta_0$) is **40.85**, this means that when **temperature** is 0 **(degrees, fahrenheit)**, **visits** is on average **41 people**. This is statistically **significant**, meaning that it is different from **the average visits** in the population.

The coefficient ($\beta_1$) for ** visits ** is **.468 **. This means that for every ** 1 degree ** increase in ** temp **, **visits ** **increases (choose one)** by **.468**. This is statistically **significant**, meaning that it is different from **random visits** in the population. In other words, we can reject the null hypothesis that there is no effect of temperature on campus foot traffic.  **Additionally, because the P> |t| is .003, this means that there is a less than 1 percent chance that the temp variable has no effect on the dependent variable, visits.** 

# 2. Avocado price and demand (5 points) (multiple regression)

For this exercise, use our familiar avocado dataset.

For the following cities, run a multiple regression that investigates the relationship between demand and price while simultaneously accounting for the effect of regions.

Use only the conventional avocado data from the following cities:

```
cities = ['Philadelphia', 'NewYork', 'BaltimoreWashington', 'Boston', 'Chicago', 'SanFrancisco']
```


In [16]:
# read in the dataset, call it avocado
avocado = pandas.read_csv('avocado.csv', index_col = 0)
avocado

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.00,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.00,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.00,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.00,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.00,conventional,2015,Albany
5,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.00,conventional,2015,Albany
6,2015-11-15,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05,0.00,conventional,2015,Albany
7,2015-11-08,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.00,conventional,2015,Albany
8,2015-11-01,1.02,99811.42,1022.15,87315.57,85.34,11388.36,11104.53,283.83,0.00,conventional,2015,Albany
9,2015-10-25,1.07,74338.76,842.40,64757.44,113.00,8625.92,8061.47,564.45,0.00,conventional,2015,Albany


In [17]:
# select only the rows corresponding to the cities listed above
# call the resulting dataframe avocado (replace avocado with the result of this subset selection)
# use .isin(...) method (see notes from module 4 bar plot to find an example of this)
# also make sure to select only the data for the conventional avocado type
cities = ['Philadelphia', 'NewYork', 'BaltimoreWashington', 'Boston', 'Chicago', 'SanFrancisco']
avocado = avocado.loc[avocado['region'].isin(cities)] 
avocado = avocado.loc[avocado['type'] == 'conventional']
avocado


,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
104,2015-12-27,1.17,596819.40,40450.49,394104.02,17353.79,144911.10,142543.88,2367.22,0.00,conventional,2015,BaltimoreWashington
105,2015-12-20,1.11,642682.40,44741.37,434590.82,19112.14,144238.07,141751.91,2486.16,0.00,conventional,2015,BaltimoreWashington
106,2015-12-13,1.15,619509.33,44400.26,399851.59,24291.20,150966.28,149070.40,1895.88,0.00,conventional,2015,BaltimoreWashington
107,2015-12-06,1.04,649141.25,51129.29,453586.50,25760.49,118664.97,117112.77,1552.20,0.00,conventional,2015,BaltimoreWashington
108,2015-11-29,1.16,545800.05,41028.15,379156.09,20147.53,105468.28,103615.95,1852.33,0.00,conventional,2015,BaltimoreWashington
109,2015-11-22,1.12,572019.80,40867.80,386137.78,23343.49,121670.73,119007.61,2663.12,0.00,conventional,2015,BaltimoreWashington
110,2015-11-15,1.04,718330.48,47109.75,503470.08,37990.29,129760.36,125297.67,4462.69,0.00,conventional,2015,BaltimoreWashington
111,2015-11-08,1.05,830967.23,58579.27,576512.71,51035.04,144840.21,138777.63,6062.58,0.00,conventional,2015,BaltimoreWashington
112,2015-11-01,1.12,742220.67,51860.07,473819.14,58615.76,157925.70,152716.40,5209.30,0.00,conventional,2015,BaltimoreWashington
113,2015-10-25,1.19,656892.03,53766.25,397911.35,49085.74,156128.69,149987.55,6141.14,0.00,conventional,2015,BaltimoreWashington


In [18]:
# show the unique regions in the resulting dataframe 
# make sure we have only the ones we wanted to select from above

avocado['region'].unique()results



array(['BaltimoreWashington', 'Boston', 'Chicago', 'NewYork',
       'Philadelphia', 'SanFrancisco'], dtype=object)

In [19]:
# show the unique types in the resulting dataframe 
# make sure we have only conventional

avocado['type'].unique()


array(['conventional'], dtype=object)

**edit this cell**
To model the demand of avocaldos, we want to run the following regression:

$$
Demand = \beta_0 + \beta_1 Price +\beta_2 Boston +\beta_3Chicago +\beta_4NewYork + \beta_5 Philadelphia + \beta_6 SanFrancisco +e
$$

In this equation (given the cities we've selected for the dataset), $\beta_0$ represents the average demand in **in each city** when price is \$ $0$.



In [20]:
# since statsmodels expects variable names to not have spaces
# use a list comprehension to replace  spaces with underscores
# make sure you understand how/why this works:
avocado.columns = [c.replace(' ', '') for c in avocado.columns]
avocado

,Date,AveragePrice,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,year,region
104,2015-12-27,1.17,596819.40,40450.49,394104.02,17353.79,144911.10,142543.88,2367.22,0.00,conventional,2015,BaltimoreWashington
105,2015-12-20,1.11,642682.40,44741.37,434590.82,19112.14,144238.07,141751.91,2486.16,0.00,conventional,2015,BaltimoreWashington
106,2015-12-13,1.15,619509.33,44400.26,399851.59,24291.20,150966.28,149070.40,1895.88,0.00,conventional,2015,BaltimoreWashington
107,2015-12-06,1.04,649141.25,51129.29,453586.50,25760.49,118664.97,117112.77,1552.20,0.00,conventional,2015,BaltimoreWashington
108,2015-11-29,1.16,545800.05,41028.15,379156.09,20147.53,105468.28,103615.95,1852.33,0.00,conventional,2015,BaltimoreWashington
109,2015-11-22,1.12,572019.80,40867.80,386137.78,23343.49,121670.73,119007.61,2663.12,0.00,conventional,2015,BaltimoreWashington
110,2015-11-15,1.04,718330.48,47109.75,503470.08,37990.29,129760.36,125297.67,4462.69,0.00,conventional,2015,BaltimoreWashington
111,2015-11-08,1.05,830967.23,58579.27,576512.71,51035.04,144840.21,138777.63,6062.58,0.00,conventional,2015,BaltimoreWashington
112,2015-11-01,1.12,742220.67,51860.07,473819.14,58615.76,157925.70,152716.40,5209.30,0.00,conventional,2015,BaltimoreWashington
113,2015-10-25,1.19,656892.03,53766.25,397911.35,49085.74,156128.69,149987.55,6141.14,0.00,conventional,2015,BaltimoreWashington


In [21]:
# take a look at the resulting dataframe

avocado.head()

,Date,AveragePrice,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,year,region
104,2015-12-27,1.17,596819.40,40450.49,394104.02,17353.79,144911.10,142543.88,2367.22,0.0,conventional,2015,BaltimoreWashington
105,2015-12-20,1.11,642682.40,44741.37,434590.82,19112.14,144238.07,141751.91,2486.16,0.0,conventional,2015,BaltimoreWashington
106,2015-12-13,1.15,619509.33,44400.26,399851.59,24291.20,150966.28,149070.40,1895.88,0.0,conventional,2015,BaltimoreWashington
107,2015-12-06,1.04,649141.25,51129.29,453586.50,25760.49,118664.97,117112.77,1552.20,0.0,conventional,2015,BaltimoreWashington
108,2015-11-29,1.16,545800.05,41028.15,379156.09,20147.53,105468.28,103615.95,1852.33,0.0,conventional,2015,BaltimoreWashington


In [114]:
##regions = {'b_region': 'Boston', 'bw_region': 'BaltimoreWashington', 'c_region': 'Chicago', 'n_region': 'NewYork', 'p_region': 'Philadelphia', 's_region': 'SanFrancisco'}
##for key in regions:
##    avocado[key] = 0  
##    avocado.loc[avocado['region'] == regions[key], key] = 1
##avocado
#regions = ['b_region': 'Boston', 'bw_region': 'BaltimoreWashington', 'c_region': 'Chicago', 'n_region': 'NewYork', 'p_region': 'Philadelphia', 's_region': 'SanFrancisco'}
##for key in regions:
##    avocado[key] = 0  
##    avocado.loc[avocado['region'] == regions[key], key] = avocado['AveragePrice']
##avocado


,Date,AveragePrice,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,year,region,b_region,bw_region,c_region,n_region,p_region,s_region
104,2015-12-27,1.17,596819.40,40450.49,394104.02,17353.79,144911.10,142543.88,2367.22,0.00,conventional,2015,BaltimoreWashington,0.0,1.17,0.0,0.00,0.00,0.00
105,2015-12-20,1.11,642682.40,44741.37,434590.82,19112.14,144238.07,141751.91,2486.16,0.00,conventional,2015,BaltimoreWashington,0.0,1.11,0.0,0.00,0.00,0.00
106,2015-12-13,1.15,619509.33,44400.26,399851.59,24291.20,150966.28,149070.40,1895.88,0.00,conventional,2015,BaltimoreWashington,0.0,1.15,0.0,0.00,0.00,0.00
107,2015-12-06,1.04,649141.25,51129.29,453586.50,25760.49,118664.97,117112.77,1552.20,0.00,conventional,2015,BaltimoreWashington,0.0,1.04,0.0,0.00,0.00,0.00
108,2015-11-29,1.16,545800.05,41028.15,379156.09,20147.53,105468.28,103615.95,1852.33,0.00,conventional,2015,BaltimoreWashington,0.0,1.16,0.0,0.00,0.00,0.00
109,2015-11-22,1.12,572019.80,40867.80,386137.78,23343.49,121670.73,119007.61,2663.12,0.00,conventional,2015,BaltimoreWashington,0.0,1.12,0.0,0.00,0.00,0.00
110,2015-11-15,1.04,718330.48,47109.75,503470.08,37990.29,129760.36,125297.67,4462.69,0.00,conventional,2015,BaltimoreWashington,0.0,1.04,0.0,0.00,0.00,0.00
111,2015-11-08,1.05,830967.23,58579.27,576512.71,51035.04,144840.21,138777.63,6062.58,0.00,conventional,2015,BaltimoreWashington,0.0,1.05,0.0,0.00,0.00,0.00
112,2015-11-01,1.12,742220.67,51860.07,473819.14,58615.76,157925.70,152716.40,5209.30,0.00,conventional,2015,BaltimoreWashington,0.0,1.12,0.0,0.00,0.00,0.00
113,2015-10-25,1.19,656892.03,53766.25,397911.35,49085.74,156128.69,149987.55,6141.14,0.00,conventional,2015,BaltimoreWashington,0.0,1.19,0.0,0.00,0.00,0.00


In [26]:
# run the regression using statsmodels
# call the result object res
# for demand, use 'Total Bags'

res = smf.ols('TotalBags ~ AveragePrice + region', data = avocado).fit()

In [40]:
# print the regression summary table
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              TotalBags   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     546.7
Date:                Tue, 19 Oct 2021   Prob (F-statistic):          1.30e-312
Time:                        22:02:21   Log-Likelihood:                -12529.
No. Observations:                1014   AIC:                         2.507e+04
Df Residuals:                    1007   BIC:                         2.511e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               1.847e+05   1.05e+04     17.671      0.000    1.64e+05    2.05e+05
region[T.Boston]       -8.097e+04   6144.983    -13.177      0.000    -9.3e+04   -6.89e+04
region[T.Chicago]      -9.941e+04   6141.100    -16.188      0.000   -1.11e+05   -8.74e+04
region[T.NewYork]       1.843e+05   6152.184     29.961      0.000    1.72e+05    1.96e+05
region[T.Philadelphia]  -6.29e+04   6149.588    -10.229      0.000    -7.5e+04   -5.08e+04
region[T.SanFrancisco] -1.097e+05   6149.942    -17.839      0.000   -1.22e+05   -9.76e+04
AveragePrice            8121.8105   7073.444      1.148      0.251   -5758.568     2.2e+04
==============================================================================
Omnibus:                      586.949   Durbin-Watson:                   0.765
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10971.908
Skew:                           2.262   Prob(JB):                         0.00
Kurtosis:                      18.467   Cond. No.                         12.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [78]:
model = smf.ols('TotalBags ~ AveragePrice + region', data = avocado).fit()

In [81]:
params = model.params
params['AveragePrice']

-405297.78276384296

**Edit this cell**
1. The R$^2$ is **.765**, this means that **77%**\% of the variation in demand is explained by the model.
1. The intercept ($\beta_0$) is **_184,700** and is statistically **significant **. This is the average demand in **_Total Bags_** when price is \$0.
1. The coefficient for Boston is **_essentially zero_**, which means that the average demand in Boston is **_____slightly _** **fewer** bags than in **__on average_**. This difference is statistically **insignificant (pick one)**
1. The coefficient for price is **_8121.81_**, which means that when price increases by \$1 in any region, demand increases by on average **____8122_______** bags. This effect is statistically **significant** which means that there is enough evidence to show that price **does ** have a positive** impact on demand.
1. The region with the highest demand is **_New York_**.

# 3. Putting what we've learned all together (the power of programming) (2 points - extra point for bonus, try your best - there may or may not be a prize involved)

In this exercise, you will run one regression for each region and each avocado type, i.e. one regression of total bags sold on price for Philly conventional, one for Philly organic, one for NY conventional, one for NY organic, and so on... (think loop). 

You will then visualize the coefficients for conventional and organic price by region on a scatter plot (each dot's (x, y) coordinates = one region's (price coefficient for conventional, price coefficient for organic). 

To do this break down the problem to the following components (and then put it all together):

* Suppose you have selected a region $r$ and avocado type $t$, how would you run the regression?
* How can you extract just the price coefficient given a regression result object `res`?
* For some region, `r`, and some avocado type `t`, how would you select only the rows pertaining to region `r` and type `t`?
* How can you loop through regions and types? (Need 2 nested "for" loops, one inside the other)
* Suppose you have named the conventional and organic price coefficients for region `r` as `bConv` and `bOrg`, how would you create a list that looks like: `[region, conventional coeff, organic coefficient]`?
* Given a list `results`, how would you *append* the list above to `results`? i.e. make keep adding to `results`: `[[region1, conventional coeff1, organic coeff1], [region2, conventional coeff2, organic coeff2], [region3, conventional coeff3, organic coeff3],...]`
* Once you've run every regression and have a complete `results` list, convert it to a dataframe using `df = pd.DataFrame(results)`, give it some column names.

The final code might have a structure similar to the pseudo code below:

```
results = list()
for r in ##all the unique regions##:
    for t in ##all the unique types##:
        temp = # Select the appropriate rows of df
        # run the regression using temp
        if ##this iteration represents conventional avocados##:
            ##extract the price parameter assign it to the variable bConv##
        else:
            ##extract the price parameter and name it bOrg##
    ##append the results with newest [region, conventional price coeff, organic price coefficient]##
df = pd.DataFrame(results) # convert to dataframe
df.columns = ['region', 'beta_conv', 'beta_org'] # rename dataframe columns
```

The resulting dataframe of coefficients should look something like:

| region | beta_conv | beta_org |
|-|-|-|
| Albany |8.579340e+02	 |-3168.491384|
| Atlanta| -3.377365e+05 | -6352.937420 |
| BaltimoreWashington | -3.251570e+05 | -15537.003211 |
|...|...|...|...|

(note numbers above are illustrative, not necessarily representative of actual results)


In [28]:
# first re-read the original avocado dataframe to use for this problem
avocado = pandas.read_csv('avocado.csv', index_col = 0)

In [30]:
# rename columns to get rid of spaces
avocado.columns = [c.replace(' ', '') for c in avocado.columns]
avocado.head(2)

,Date,AveragePrice,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany


In [34]:
unique_regions = avocado['region'].unique()
unique_regions
unique_types = avocado['type'].unique()
unique_types

array(['conventional', 'organic'], dtype=object)

In [83]:
# run all the regressions
# create a dataframe with region, beta_conv, beta_org as the 3 columns 
# using the regression results
results = []

for a_region in unique_regions: 
    beta_conv = 'na'
    beta_org = 'na'
    for kind in unique_types:
        temp_df = avocado.loc[(avocado.region == a_region)]
        temp_df = temp_df.loc[(temp_df.type == kind)]
        model = smf.ols('TotalBags ~ AveragePrice + region', data = temp_df).fit()
        params = model.params
        ##beta_key = 'region[T.' + a_region + ']'
        average_price = params['AveragePrice']
        ##print(beta)
        if kind == 'conventional':
            ##beta_conv = model.rsquared
            beta_conv = average_price
        if kind == 'organic':
            ##beta_org = model.rsquared
            beta_org = average_price
    temp_list = [a_region, beta_conv, beta_org]
    results.append(temp_list)


    
df = pandas.DataFrame(results) # convert to dataframe
df.columns = ['region', 'beta_conv', 'beta_org'] # rename dataframe columns
df

,region,beta_conv,beta_org
0,Albany,8.579340e+02,-3168.491384
1,Atlanta,-8.373534e+04,-5041.977163
2,BaltimoreWashington,-1.472740e+03,-14715.844065
3,Boise,-2.862924e+04,-1438.258267
4,Boston,7.050460e+03,-369.330085
5,BuffaloRochester,-7.171370e+04,-6817.766138
6,California,-8.462063e+05,-19414.846809
7,Charlotte,2.788339e+03,-1339.038227
8,Chicago,-2.209711e+04,2780.475253
9,CincinnatiDayton,-3.001482e+04,-12285.334723


In [ ]:
# Get rid of the TotalUS region from the regression results dataframe



In [ ]:
# show the full resulting table



In [ ]:
# make scatter plot of the coefficients
# change aesthetics so the dots are not a single blob




In [ ]:
# which regions have upward sloping demand curve for organic (as price increases, so did demand)?
# use code to show these rows of df



In [ ]:
# which regions have upward sloping demand curve for conventional (as price increases, so did demand)?
# use code to show these rows of df



Why might the relationship between price and demand be upward sloping instead of the expected downward sloping demand curve theorized in economics? i.e. the law of demand, when price increases, demand decreases. 

**Your answer here**